In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import time
from tqdm import tqdm
import re

# 0.1. Rutas

In [2]:
txt_SSI_output = '../Datos/Resultados_SSI.txt'
txt_Infobra_output = '../Datos/Resultados_Infobra.txt'
txt_ubicacion_output = '../Datos/Resultados_ubicacion.txt'
txt_avance_output = '../Datos/Resultados_avance.txt'
txt_modificaciones_output = '../Datos/Resultados_Modificaciones.txt'
txt_ExpTecnico_output = '../Datos/Resultados_ExpTecnico.txt'
txt_detalleExpTecnico_output = '../Datos/Resultados_detalleExpTecnico.txt'

# 0.2. Campos

In [3]:
camposInfobras = ['NOBR_ID', 'COBR_DESCRI', 'CODCONSUCODE', 'COBR_DIRECCION', 'NOBR_TIPEJE', 'COPA_DESCRI', 'RUBRO', 'CODCONSUCODE_DESCRI', 'COBR_NOMBRERESIDENTE', 'COBR_DNIRESIDENTE', 'DOBR_FEC_INI_OBRA',
                 'NOBR_CODSNIP', 'DSNP_FECVERI', 'NSNP_MONTOVERI', 'DSNP_FECAPR', 'NSNP_MONTOAPR', 'CSNP_CODSIAF', 'CSNP_DESCRI', 'CODIGO_UNICO', 'FECHA_ACTUALIZACION']
# No encuentro el "ESTADO DE LA OBRA"

camposUbicacion = ['CODIGO_UNICO', 'NCOR_LONGIT', 'NCOR_LATITU', 'NCOR_LONGI2', 'NCOR_LATIT2', 'CDPT_CD_DPTO', 'CDPT_DPTO', 'CDPT_DPTO', 'CPRV_PROV', 'CDST_CODIGO', 'CDST_DISTRIT']
columnasAvance = ['NOBA_ANIO', 'NOBR_ID', 'DOBA_FCHREP', 'NOBA_FISICO_PRO', 'NOBA_FISICO_REA', 'NOBA_VALZDO_PRO', 'NOBA_VALZDO_REA', 'NOBA_MES', 'CODIGO_UNICO']
columnasSSI = ['CODIGO_UNICO', 'UNIDAD_OPMI', 'SECTOR', 'ENTIDAD', 'FUNCION', 'PIM_ANO_VIGENTE', 'DEV_ANO_VIGENTE', 'DEV_ACUMULADO', 'NIVEL', 'COD_UNIDAD_UEI', 'DES_UNIDAD_UEI', 'DES_UNIDAD_UF', 
               'COSTO_ACTUALIZADO', 'MTO_VIABLE', 'NOMBRE_INVERSION', 'FEC_REGISTRO', 'FEC_VIABLE', 'TIPO_FORMATO', 'MES_ANO_PRI_DEV', 'MES_ANO_ULT_DEV', 'CERTIFICADO_ANO_ACTUAL', 
               'GIRADO_ANO_ACTUAL', 'COMPROMETIDO_ANO_ACTUAL', 'MTO_PMI_1', 'MTO_PMI_2', 'MTO_PMI_3', 'MTO_PMI_4', 'MTO_CartFza', 'NOMBRE_OPMI', 'ESTADO', 'FECHA_ET', 'CIERRE_REGISTRADO', 
               'FEC_INI_EJ', 'FEC_FIN_EJ', 'MODAL_EJEC', 'MARCO', 'COD_SNIP', 'IND_REG_PMI', 'DES_PROGRAMA', 'DES_SUB_PROGRAMA', 'SITUACION', 'BENEFICIARIO', 'COD_TIPO_INVERSION', 'ID_PROYECTO']

# 0.3. Funciones

In [41]:
def unix_to_date(unix):
    unix = str(unix)
    if 'Date' in unix:
        return pd.to_datetime(re.findall(r'\b\d+\b', unix)[0][:13], unit='ms', origin='unix')
    else:
        return None

In [229]:
unix_to_date('/Date(1511499600000)/')

Timestamp('2017-11-24 05:00:00')

# 1. Universo obras

### Universo

In [235]:
dfInfobra = pd.read_csv(txt_Infobra_output, sep='|', on_bad_lines='skip')

In [240]:
dfInfobra = pd.read_csv(txt_Infobra_output, sep='|', on_bad_lines='skip')
dfInfobra = dfInfobra[camposInfobras].copy()

In [241]:
dfInfobra_revisar = dfInfobra[(dfInfobra['NOBR_ID'].fillna(0) == 0)|(dfInfobra['CODIGO_UNICO'].fillna(0) == 0)].copy()
dfInfobra = dfInfobra[(dfInfobra['NOBR_ID'].fillna(0) != 0)&(dfInfobra['CODIGO_UNICO'].fillna(0) != 0)].drop_duplicates().copy()

In [242]:
# Fecha de inicio

dfInfobra['FECHA_INICIO'] = dfInfobra['DOBR_FEC_INI_OBRA'].apply(lambda x: unix_to_date(x))

### Universo

In [249]:
dfUbicacion = pd.read_csv(txt_ubicacion_output, sep='|')[camposUbicacion].drop_duplicates().copy()

In [250]:
dfUbicacion.head()

,CODIGO_UNICO,NCOR_LONGIT,NCOR_LATITU,NCOR_LONGI2,NCOR_LATIT2,CDPT_CD_DPTO,CDPT_DPTO,CDPT_DPTO,CPRV_PROV,CDST_CODIGO,CDST_DISTRIT
0,2192666,-72.952480,-3.450366,-73.067665,-3.549744,16.0,LORETO,LORETO,MAYNAS,6.0,MAZAN
1,2506945,-75.144327,-3.811037,0.000000,0.000000,16.0,LORETO,LORETO,LORETO,4.0,TROMPETEROS
2,2015456,-73.256000,-3.753000,-73.256000,-3.752000,16.0,LORETO,LORETO,MAYNAS,1.0,IQUITOS
3,2016888,-75.009900,-7.347090,0.000000,0.000000,16.0,LORETO,LORETO,UCAYALI,1.0,CONTAMANA
4,2017024,-70.475235,-3.907414,0.000000,0.000000,16.0,LORETO,LORETO,MARISCAL RAMON CASTILLA,3.0,YAVARI


In [251]:
dfUbicacion.shape[0], dfUbicacion['CODIGO_UNICO'].nunique()

(15819, 15819)

### Avance

In [253]:
dfAvance = pd.read_csv(txt_avance_output, sep='|', on_bad_lines='skip')[columnasAvance].copy()

In [254]:
dfAvance['DOBA_FCHREP'] = dfAvance['DOBA_FCHREP'].apply(lambda x: unix_to_date(x))

In [256]:
dfAvance_agg = dfAvance.groupby('CODIGO_UNICO').agg(
                                            {
                                            'DOBA_FCHREP': ['count', 'min', 'max'],
                                            'NOBA_FISICO_PRO': ['min', 'max'],
                                            'NOBA_FISICO_REA': ['min', 'max'],
                                            'NOBA_VALZDO_PRO': ['min', 'max'],
                                            'NOBA_VALZDO_REA': ['min', 'max']
                                            }
                                        )

In [257]:
dfAvance_agg.columns = [x[0]+'_'+x[1] for x in dfAvance_agg.columns]
dfAvance_agg.reset_index(inplace=True)

dfAvance_agg['CODIGO_UNICO'] = dfAvance_agg['CODIGO_UNICO'].apply(lambda x: int(x))

In [258]:
dfAvance_agg.head()

,CODIGO_UNICO,DOBA_FCHREP_count,DOBA_FCHREP_min,DOBA_FCHREP_max,NOBA_FISICO_PRO_min,NOBA_FISICO_PRO_max,NOBA_FISICO_REA_min,NOBA_FISICO_REA_max,NOBA_VALZDO_PRO_min,NOBA_VALZDO_PRO_max,NOBA_VALZDO_REA_min,NOBA_VALZDO_REA_max
0,2015456,1,2013-02-19 21:30:20,2013-02-19 21:30:20,3.50,3.50,0.00,0.00,24424.41,24424.41,0.00,0.00
1,2017024,14,2017-02-02 13:52:07,2017-02-06 16:51:04,6.81,100.00,18.38,54.40,51669.28,758434.70,139399.60,412558.11
2,2017105,1,2015-11-19 21:25:19,2015-11-19 21:25:19,71.98,71.98,71.98,71.98,260809.79,260809.79,260809.79,260809.79
3,2017492,10,2012-10-22 18:22:42,2022-03-25 20:25:25,7.39,99.90,7.39,99.90,63677.15,1628563.96,63677.15,1628563.96
4,2018052,3,2013-10-31 17:50:43,2013-10-31 19:05:02,100.00,100.00,51.81,100.00,2292654.09,2292654.09,1187860.71,2292654.09


# 2. SSI

In [330]:
def formato_fecha(fecha, formato):
    
    try:
        return pd.to_datetime(fecha, format=formato)
    except:
        return None

In [331]:
dfSSI = pd.read_csv(txt_SSI_output, sep='|', on_bad_lines='skip')

In [332]:
dfSSI_revisar = dfSSI[dfSSI['CODIGO_UNICO'].fillna('0')=='0'].copy()
dfSSI = dfSSI[dfSSI['CODIGO_UNICO'].fillna('0')!='0'][columnasSSI].copy()

In [333]:
dfSSI['RN'] = dfSSI.sort_values(['ENTIDAD', 'FUNCION'], ascending=[True,False]) \
             .groupby(['CODIGO_UNICO']) \
             .cumcount() + 1

In [334]:
dfSSI = dfSSI[dfSSI['RN'] == 1].drop('RN', axis=1).copy()

In [336]:
for columna in ['FEC_REGISTRO', 'FEC_VIABLE', 'FECHA_ET']:
    dfSSI[columna] = dfSSI[columna].apply(lambda x: unix_to_date(x))

In [337]:
for columna in ['FEC_INI_EJ', 'FEC_FIN_EJ']:
    dfSSI[columna] = dfSSI[columna].apply(lambda x: formato_fecha(x, '%d/%m/%Y %H:%M:%S'))# pd.to_datetime(dfSSI[columna], format='%d/%m/%Y %H:%M:%S')

In [339]:
dfSSI[['CODIGO_UNICO', 'MTO_VIABLE', 'COSTO_ACTUALIZADO']].head()

,CODIGO_UNICO,MTO_VIABLE,COSTO_ACTUALIZADO
0,2192666,1.128900e+09,1.542915e+09
1,2506945,2.033026e+05,2.008549e+05
2,2001883,8.125250e+06,8.125250e+06
3,2002163,3.250000e+06,3.250000e+06
4,2002584,7.244700e+05,7.244700e+05


### Variables

In [340]:
dfSSI['DIFF_MONTO'] = dfSSI['COSTO_ACTUALIZADO'] - dfSSI['MTO_VIABLE']
dfSSI['DIFF_PORC_MONTO'] = (dfSSI['COSTO_ACTUALIZADO'] / dfSSI['MTO_VIABLE']) - 1

In [341]:
dfSSI['NOW'] = pd.to_datetime("now") 
dfSSI['DIFF_FECHA_INI_EJE_ET'] = dfSSI['FEC_INI_EJ'].dt.to_period('M').astype(int) - dfSSI['FECHA_ET'].dt.to_period('M').astype(int)
dfSSI['DIFF_FECHA_NOW_ET'] = dfSSI['NOW'].dt.to_period('M').astype(int) - dfSSI['FECHA_ET'].dt.to_period('M').astype(int)
dfSSI['DIFF_FECHA_NOW_INI_EJE'] = dfSSI['NOW'].dt.to_period('M').astype(int) - dfSSI['FEC_INI_EJ'].dt.to_period('M').astype(int)

In [342]:
for columna in ['DIFF_FECHA_NOW_ET', 'DIFF_FECHA_NOW_INI_EJE']:
    dfSSI[columna] = dfSSI[columna].apply(lambda x: x if x<100_000 else None)
    dfSSI[columna] = dfSSI[columna].apply(lambda x: x if x>-100_000 else None)

In [343]:
dfSSI.shape, dfSSI['CODIGO_UNICO'].nunique()

((52275, 50), 52275)

In [320]:
dfSSI.head()

,CODIGO_UNICO,UNIDAD_OPMI,SECTOR,ENTIDAD,FUNCION,PIM_ANO_VIGENTE,DEV_ANO_VIGENTE,DEV_ACUMULADO,NIVEL,COD_UNIDAD_UEI,DES_UNIDAD_UEI,DES_UNIDAD_UF,COSTO_ACTUALIZADO,MTO_VIABLE,NOMBRE_INVERSION,FEC_REGISTRO,FEC_VIABLE,TIPO_FORMATO,MES_ANO_PRI_DEV,MES_ANO_ULT_DEV,CERTIFICADO_ANO_ACTUAL,GIRADO_ANO_ACTUAL,COMPROMETIDO_ANO_ACTUAL,MTO_PMI_1,MTO_PMI_2,MTO_PMI_3,MTO_PMI_4,MTO_CartFza,NOMBRE_OPMI,ESTADO,FECHA_ET,CIERRE_REGISTRADO,FEC_INI_EJ,FEC_FIN_EJ,MODAL_EJEC,MARCO,COD_SNIP,IND_REG_PMI,DES_PROGRAMA,DES_SUB_PROGRAMA,SITUACION,BENEFICIARIO,COD_TIPO_INVERSION,ID_PROYECTO,DIFF_MONTO,DIFF_PORC_MONTO,NOW,DIFF_FECHA_INI_EJE_ET,DIFF_FECHA_NOW_ET,DIFF_FECHA_NOW_INI_EJE
0,2192666,0,AGRICULTURA Y RIEGO,MINISTERIO DE AGRICULTURA Y RIEGO - MINAGRI,TRANSPORTE,11114703.0,9563.78,671613901.36,GN,39183,dirección de puentes de pvn,PROYECTO ESPECIAL BINACIONAL DESARROLLO INTEGR...,1.542915e+09,1.128900e+09,CONSTRUCCION DE LA CARRETERA BELLAVISTA - MAZA...,2001-07-26 09:57:33,2014-06-18 05:00:00,PROYECTO DE INVERSION,2014-05,2022-03,6627862.76,9563.78,12923.78,2988777.0,42023766.0,0.0,0,0.0,OPMI DEL MINISTERIO DE AGRICULTURA - MINAG,ACTIVO,2015-04-30 05:00:00,NO,2017-07-18,2024-03-27,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,SNIP,396.0,SI,TRANSPORTE TERRESTRE,CONSTRUCCION Y MEJORAMIENTO DE CARRETERAS,VIABLE,121641.0,0.0,0.0,4.140149e+08,0.366742,2022-05-06 05:30:53.794666,27,85.0,58.0
1,2506945,22403,DESARROLLO E INCLUSION SOCIAL,MINISTERIO DE DESARROLLO E INCLUSION SOCIAL - ...,TRANSPORTE,0.0,0.0,0.0,GN,27634,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,2.008549e+05,2.033026e+05,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,2020-12-15 03:04:42,2020-12-16 05:10:55,IOARR,NaN,NaN,0.00,0.00,0.00,0.0,0.0,0.0,0,0.0,OPMI DEL MINISTERIO DE DESARROLLO E INCLUSION ...,ACTIVO,2021-07-21 05:00:00,NO,2021-01-25,2021-11-07,ADMINISTRACIÓN INDIRECTA - NÚCLEO EJECUTOR,INVIERTE,2506945.0,NO,TRANSPORTE TERRESTRE,CAMINOS DE HERRADURA,APROBADO,0.0,2.0,117416.0,-2.447720e+03,-0.012040,2022-05-06 05:30:53.794666,-6,10.0,16.0
2,2001883,0,AGRICULTURA Y RIEGO,INSTITUTO NACIONAL DE RECURSOS NATURALES - INRENA,AGRARIA,0.0,0.0,28152498.7,GN,NaN,NaN,INSTITUTO NACIONAL DE RECURSOS NATURALES,8.125250e+06,8.125250e+06,PARTICIPACIÓN DE LAS COMUNIDADES NATIVAS EN E...,2003-12-16 18:26:43,2004-03-25 05:00:00,PROYECTO DE INVERSION,2004-01,2007-09,0.00,0.00,0.00,0.0,0.0,0.0,0,0.0,OPMI DEL MINISTERIO DE AGRICULTURA - MINAG,ACTIVO,NaT,NO,NaT,NaT,---,SNIP,8117.0,NO,PRESERVACION DE LOS RECURSOS NATURALES RENOVABLES,PROTECCION DE LA FLORA Y FAUNA,VIABLE,0.0,0.0,0.0,0.000000e+00,0.000000,2022-05-06 05:30:53.794666,0,NaN,NaN
3,2002163,0,AGRICULTURA Y RIEGO,INSTITUTO NACIONAL DE RECURSOS NATURALES - INRENA,AGRARIA,0.0,0.0,5303185.32,GN,NaN,NaN,INSTITUTO NACIONAL DE RECURSOS NATURALES,3.250000e+06,3.250000e+06,APOYO A LA IMPLEMENTACION DEL FONDO DE PROMOCI...,2002-04-02 02:03:08,2002-09-18 17:20:19,PROYECTO DE INVERSION,2004-12,2006-12,0.00,0.00,0.00,0.0,0.0,0.0,0,0.0,OPMI DEL MINISTERIO DE AGRICULTURA - MINAG,ACTIVO,NaT,NO,NaT,NaT,---,SNIP,2025.0,NO,PRESERVACION DE LOS RECURSOS NATURALES RENOVABLES,PROTECCION DE LA FLORA Y FAUNA,VIABLE,0.0,0.0,0.0,0.000000e+00,0.000000,2022-05-06 05:30:53.794666,0,NaN,NaN
4,2002584,0,INTERIOR,MINISTERIO DEL INTERIOR,DEFENSA Y SEGURIDAD NACIONAL,0.0,0.0,306395.06,GN,NaN,NaN,MINISTERIO DEL INTERIOR - DIRECCION GENERAL DE...,7.244700e+05,7.244700e+05,CONSTRUCCION E IMPLEMENTACION DEL PUESTO DE VI...,2002-10-23 20:51:01,2002-11-13 14:38:05,PROYECTO DE INVERSION,2007-12,2011-04,0.00,0.00,0.00,0.0,0.0,0.0,0,0.0,OPMI DEL MINISTERIO DEL INTERIOR,ACTIVO,NaT,NO,NaT,NaT,---,SNIP,3852.0,NO,ORDEN INTERNO,OPERACIONES POLICIALES,VIABLE,0.0,0.0,0.0,0.000000e+00,0.000000,2022-05-06 05:30:53.794666,0,NaN,NaN


### Modificaciones

In [273]:
dfModificaciones = pd.read_csv('../Datos/Resultados_Modificaciones_temp.txt', sep='\t', encoding='latin-1')

In [274]:
dfModificaciones.head()

,Fecha de última modificación,Monto actualizado S/,Comentarios,Usuario,Tipo de documento,¿Es histórico?,Ver,CODIGO_UNICO,FECHA_ACTUALIZACION
0,22/09/2021 03:49:47 p.m.,1.542915e+09,MODIFICACIONES EN LA FASE DE EJECUCION - INVIERTE,UF: AGR-PEDICP / UEI: UEIMTC008,NaN,NO,NaN,2192666,2022-04-24
1,09/11/2015 10:43:14 a.m.,1.497196e+09,NaN,OPITRANS,ACT. PRECIOS,NO,NaN,2192666,2022-04-24
2,07/05/2015 04:06:50 p.m.,6.449693e+08,NaN,OPITRANS,(F15),NO,NaN,2192666,2022-04-24
3,30/04/2015 12:00:00 a.m.,1.481577e+09,NaN,NaN,(F17),NO,NaN,2192666,2022-04-24
4,29/09/2021 09:52:45 p.m.,2.008549e+05,MODIFICACIONES EN LA FASE DE EJECUCION - INVIERTE,UF: UF FONCODES / UEI: UEIMIDIS003,NaN,NO,NaN,2506945,2022-04-24


In [275]:
dfModificaciones_agg = dfModificaciones[dfModificaciones['Monto actualizado S/'].notnull()].groupby('CODIGO_UNICO').agg(
                                                                                                        {'Monto actualizado S/': ['count', 'min', 'max']}
                                                                                                        )

In [276]:
dfModificaciones_agg.columns = [x[0]+'_'+x[1] for x in dfModificaciones_agg.columns]
dfModificaciones_agg.reset_index(inplace=True)

In [277]:
dfModificaciones_agg.head()

,CODIGO_UNICO,Monto actualizado S/_count,Monto actualizado S/_min,Monto actualizado S/_max
0,2015188,6,31490056.43,37280740.85
1,2015456,1,1239095.80,1239095.80
2,2016822,1,7036200.00,7036200.00
3,2016888,3,3579656.25,3706501.50
4,2017024,2,836080.51,1132449.02


# 3. Contratos

In [29]:
#contratos = pd.read_csv('../Datos/Listado_Contratos.csv', sep='|').drop('NUM_ITEM', axis=1).drop_duplicates()
contratos = pd.read_csv('../Datos/Listado_Contratos.csv', sep='|').drop_duplicates()

In [30]:
contratos['RN'] = contratos.sort_values(['RUC_CONTRATISTA', 'RUC_DESTINATARIO_PAGO'], ascending=[False, False]) \
             .groupby(['N_COD_CONTRATO']) \
             .cumcount() + 1

In [32]:
columns_sum = ['MONTO_ADICIONAL', 'MONTO_REDUCCION', 'MONTO_PRORROGA', 'MONTO_COMPLEMENTARIO']

agg = {'NUM_ITEM': 'nunique'}

for col in columns_sum:
    agg[col] = 'sum'

In [38]:
contratos_agg = contratos.groupby('N_COD_CONTRATO').agg(agg).reset_index()

In [43]:
contratos_unicos = pd.merge(contratos[contratos['RN']==1].drop(columns_sum+['NUM_ITEM'], axis=1), 
                           contratos_agg,
                           how='left', on='N_COD_CONTRATO'
                           )

In [44]:
contratos_unicos.head(5)

,CODIGOCONVOCATORIA,N_COD_CONTRATO,DESCRIPCION_PROCESO,FECHA_PUBLICACION_CONTRATO,FECHA_SUSCRIPCION_CONTRATO,FECHA_VIGENCIA_INICIAL,FECHA_VIGENCIA_FINAL,FECHA_VIGENCIA_FIN_ACTUALIZADA,CODIGO_CONTRATO,NUM_CONTRATO,MONTO_CONTRATADO_TOTAL,MONTO_CONTRATADO_ITEM,URLCONTRATO,CODIGOENTIDAD,MONEDA,RUC_CONTRATISTA,RUC_DESTINATARIO_PAGO,TIENERESOLUCION,ANIO_CONTRATO,CUI,SNIP,RN,NUM_ITEM,MONTO_ADICIONAL,MONTO_REDUCCION,MONTO_PRORROGA,MONTO_COMPLEMENTARIO
0,429854,1149612,ADQUISICION DE COMBUSTIBLE PARA LAS DIFERENTES...,2018-03-23 00:00:00,2018-03-14 00:00:00,2018-03-14 00:00:00,2018-12-28 00:00:00,2018-12-28 00:00:00,001229-00429854-01149612,01-2018-MDJ,37461.99,37461.99,http://zonasegura.seace.gob.pe/documentos/mon\...,1229,Soles,10193287501,10193287501,NO,2018,NaN,NaN,1,1,0.0,0.0,0.0,0.0
1,430724,1154235,ADQUISICION DE PRODUCTOS PARA EL PROGRAMA VASO...,2018-04-26 00:00:00,2018-03-26 00:00:00,2018-03-26 00:00:00,2018-12-31 00:00:00,2018-12-31 00:00:00,000520-00430724-01154235,14,66769.00,66769.00,http://zonasegura.seace.gob.pe/documentos/mon\...,520,Soles,20287233851,20287233851,NO,2018,NaN,NaN,1,1,0.0,0.0,0.0,0.0
2,430745,1155964,CONTRATACIÓN DEL SERVICIO DE SEGURIDAD Y VIGIL...,2018-05-09 00:00:00,2018-04-10 00:00:00,2018-04-11 00:00:00,2019-04-10 00:00:00,2019-04-10 00:00:00,200642-00430745-01155964,31,67000.00,67000.00,http://zonasegura.seace.gob.pe/documentos/mon\...,200642,Soles,20601692806,20601692806,NO,2018,NaN,NaN,1,1,0.0,0.0,0.0,0.0
3,430757,1155121,CONTRATACIÓN DEL SERVICIO DE LIMPIEZA DE LOCAL...,2018-05-03 00:00:00,2018-04-11 00:00:00,2018-04-12 00:00:00,2019-04-11 00:00:00,2019-07-08 00:00:00,200642-00430757-01155121,32,34800.00,34800.00,http://zonasegura.seace.gob.pe/documentos/mon\...,200642,Soles,20554470557,20554470557,NO,2018,NaN,NaN,1,1,1509.1,0.0,0.0,0.0
4,430904,1154535,"CONTRATACION DE BIEN PAJILLA DE SEMEN, SEGUN E...",2018-04-29 00:00:00,2018-04-12 00:00:00,2018-04-13 00:00:00,2018-05-31 00:00:00,2018-05-31 00:00:00,001666-00430904-01154535,COMPARACION DE PRECIOS NRO 001-2018-MDA,55000.00,55000.00,http://zonasegura.seace.gob.pe/documentos/mon\...,1666,Soles,20447940044,20447940044,NO,2018,NaN,NaN,1,1,0.0,0.0,0.0,0.0


In [45]:
contratos_unicos.shape[0], contratos_unicos['N_COD_CONTRATO'].nunique()

(221409, 221409)

### API contratos

In [46]:
Resultados_Contratos = pd.read_csv('../Datos/Resultados_Contratos_tab.txt', sep='\t', encoding='latin-1')

In [47]:
Resultados_Contratos_2 = pd.read_csv('../Datos/Resultados_Contratos_d2.txt', sep='|', encoding='latin-1')

In [48]:
Resultados_Contratos_2.head()

,idContrato,entidadContratante,contratista,descripcionContrato,numeroContrato,rucCodigoContratista,tipoMoneda,montoContrato,fechaSuscripcionContrato,fechaInicioVigenciaContrato,fechaPublicacionContrato,fechaFinVigenciaContrato,lugar,destinatarioPago,estadoSituacional,archivoAdjunto,entidadConvocante,rucDestinoPago,nomenclaturaProceso,montoPresupuesto,montoCCP,montoPrevision,archivoAdjunto2,idDocumento2,tamanioDocumento2,fechaDocumento2,indPresupuesto,listaItemsContrato,listaGarantiaContrato,listaAccionContrato,listaProyectoContrato,listaResolucionDisputas,listaConciliaciones,listaArbitrajes,listaPresupuestoGeneral,idDocumento,tamanioDocumento,fechaDocumento,nIndConsorcio,idDocumentoConsorcio,FECHA_ACTUALIZACION
0,2107277,20153219118-HOSPITAL NACIONAL HIPOLITO UNANUE,20537758377-ICU MEDICAL PERU S.R.L.,ADQUISICIÃN DE LÃNEAS PARA BOMBAS DE INFUSIÃ...,CONTRATO NÂº010-03-2022,20561372111,Soles,233055.7,21/03/2022,22/03/2022,22/03/2022,22/06/2022,PIURA,20561372111-BIOINDUSTRIAS JB E.I.R.L.,Publicado,CONTRATO NÂº010-03-2022-MPT.pdf,20146713824-MUNICIPALIDAD PROVINCIAL DE TALARA...,20561372111,AS-SM-1-2022-CS-MPT-1,235863.6,235863.6,,,,,,1,"[{'idItem': 211904, 'nroItem': '1', 'cubso': '...",[],[],[],[],[],[],None,426758,3092929,22/03/2022 11:46 AM,0,None,2022-05-01
1,2102923,20131257750-SEGURO SOCIAL DE SALUD,20271779721-EL SUPER S.A.C.,ADQUISICIÃN DE ALIMENTOS PARA PERSONAS: EMBUT...,4503512552,20271779721,Soles,77150,05/02/2020,10/02/2020,09/03/2022,10/02/2021,LIMA,20271779721-EL SUPER S.A.C.,Publicado,4503512552.PDF,20131257750-SEGURO SOCIAL DE SALUD,20271779721,AS-SM-57-2019-ESSALUD/RAL-1,,,,,,,,0,"[{'idItem': 209605, 'nroItem': '1', 'cubso': '...",[],[],[],[],[],[],None,420184,1322833,09/03/2022 10:31 AM,0,None,2022-05-01
2,2101521,20131257750-SEGURO SOCIAL DE SALUD,20600579925-CONSORCIO MARINE,CONTRATACION DEL SERVICIO DE MANTENIMIENTO INT...,36-GRAAN-ESSALUD-2018,20600579925,Soles,164097.89,26/11/2018,27/11/2018,28/02/2022,15/03/2019,LIMA,20600579925-BUSINESS INTERNATIONAL CORE S.A.C.,Publicado,CONTRATO 036-GRAAN-ESSALUD-2018 - SERVICIO MAN...,20131257750-SEGURO SOCIAL DE SALUD,20600579925,AS-SM-31-2018-ESSALUD/RAAN-1,,,,CONTRATO DE CONSORCIO.pdf,416252,791948,25/02/2022 06:13 PM,0,"[{'idItem': 208079, 'nroItem': '1', 'cubso': '...","[{'idGarantia': 28723, 'claseGarantia': 'Garan...",[],[],[],[],"[{'demandante': 'Entidad', 'tipoArbitraje': 'I...",None,416251,1915892,25/02/2022 06:13 PM,1,None,2022-05-01
3,2101020,20163949980-MUNICIPALIDAD DISTRITAL DE TINTAY,20555116757-JESMHART INVERSIONES S.A.C.,"SERVICIO DE MOVIMIENTO CORTE, DESQUINCHE DE TA...",128,20555116757,Soles,62000,10/10/2019,16/10/2019,01/03/2022,31/10/2019,APURIMAC,20555116757-JESMHART INVERSIONES S.A.C.,Publicado,CONTRATO AROSTE.pdf,20163949980-MUNICIPALIDAD DISTRITAL DE TINTAY,20555116757,COMPRE-SM-1-2019-MDT-1,62000,62000,,,,,,1,"[{'idItem': 207556, 'nroItem': '1', 'cubso': '...",[],[],"[{'codigoUnicoInversion': '2457937', 'descripc...",[],[],[],None,415244,3017033,23/02/2022 10:59 AM,0,None,2022-05-01
4,2100784,20508353759-AUTORIDAD MUNICIPAL DE LOS PANTANO...,20605450548-RG GLOBAL PROJECT S.A.C.,Servicio de consultorÃ­a para la realizaciÃ³n ...,001-2022-MML/PROHVILLA-DT,20605450548,Soles,45785.18,02/02/2022,02/02/2022,21/02/2022,12/05/2022,LIMA,20605450548-RG GLOBAL PROJECT S.A.C.,Publicado,CONTRATO NÂ°001-2022-MML-PROHVILLA-DT 02.02.20...,20508353759-AUTORIDAD MUNICIPAL DE LOS PANTANO...,20605450548,AS-SM-2-2021-PROHVILLA-1,91570.36,91570.36,,,,,,1,"[{'idItem': 207312, 'nroItem': '1', 'cubso': '...",[],[],[],[],[],[],None,414733,2975709,21/02/2022 08:56 PM,0,None,2022-05-01


In [207]:
Resultados_Contratos_f = pd.concat([Resultados_Contratos, Resultados_Contratos_2]).drop('FECHA_ACTUALIZACION', axis=1).drop_duplicates()
Resultados_Contratos_f = Resultados_Contratos_f[Resultados_Contratos_f['idContrato'] != 'None'].copy()
Resultados_Contratos_f = Resultados_Contratos_f[Resultados_Contratos_f['idContrato'].notnull()].copy()

In [208]:
Resultados_Contratos_f['idContrato'] = Resultados_Contratos_f['idContrato'].apply(lambda x: int(float(x)))

In [211]:
import re

def get_cui(proyecto):
    
    if proyecto=='[]':
        return None
    else:
        try:
            inicio = proyecto.find('codigoUnicoInversion') + len('codigoUnicoInversion')
            
            return re.findall(r'\d+', proyecto[inicio:inicio+18])[0]
        except:
            return None

In [212]:
Resultados_Contratos_f['CUI'] = Resultados_Contratos_f['listaProyectoContrato'].apply(lambda x: get_cui(x))
Resultados_Contratos_f['arbitraje'] = Resultados_Contratos_f['listaArbitrajes'].apply(lambda x: 1 if x!='[]' else 0)
Resultados_Contratos_f['conciliacion'] = Resultados_Contratos_f['listaConciliaciones'].apply(lambda x: 1 if x!='[]' else 0)
Resultados_Contratos_f['garantia'] = Resultados_Contratos_f['listaGarantiaContrato'].apply(lambda x: 1 if x!='[]' else 0)
Resultados_Contratos_f['resolucionDisputas'] = Resultados_Contratos_f['listaResolucionDisputas'].apply(lambda x: 1 if x!='[]' else 0)

In [214]:
Resultados_Contratos_f.drop(['CUI'], axis=1).to_csv('../Datos/Matrices/Matriz_contratos.txt', sep='\t', index=False)

### CUI

In [67]:
df_cui = pd.read_csv('../Datos/Resultados_ContratosCUI.txt', sep='|').drop_duplicates()
df_cui = df_cui[df_cui['idContrato']!='2022-04-30'].copy()

In [70]:
df_cui.shape[0], df_cui['codigoUnicoInversion'].nunique(), df_cui['idContrato'].nunique()

(34131, 17276, 28825)

In [73]:
df_cui.drop(['FECHA_ACTUALIZACION'], axis=1).to_csv('../Datos/Matrices/Matriz_relacion_CUI_contratos.txt', sep='\t', index=False)

In [74]:
df_cui_temp = pd.read_csv('../Datos/Matrices/Matriz_relacion_CUI_contratos.txt', sep='\t')

# 4. OSCE: Castigos - sanciones

In [86]:
inhabilitaciones_judiciales = pd.read_csv('../Datos/OSCE/inhabilitaciones_judiciales.csv', sep='|', encoding='latin-1', dtype='str')

In [87]:
inhabilitaciones_judiciales.head(5)

,FECHA_CORTE,RUC_DNI,NOMBRE_RAZONODENOMINACIONSOCIAL,ORGANO_JURISDICCIONAL,NUMERO_RESOLUCION,FECHA_INICIO,FECHA_FIN
0,20220401,10011590921,MURGA MONTOYA JOSE ELIAS,Corte Superior de Justicia de San Martín,05,20180606,20190606
1,20220401,10015569943,MAMANI CHURA JORGE SOLON,Corte Superior de Justicia de Puno,17,20181017,20191017
2,20220401,10040039711,BARRETO MARCELO TEODORO,Corte Superior de Justicia de Pasco,SENTENCIA DE FECHA 28.04.2017,20170428,20250428
3,20220401,10086869093,ARCE JIMENEZ ERNESTO FROY,Corte Superior de Justicia de Arequipa,17,20190502,20191102
4,20220401,10107797691,LARA MALCA JUAN OSWALDO,Corte Superior de Justicia de Ancash,03,20180703,20201103


In [97]:
inhabilitaciones_judiciales.shape[0], inhabilitaciones_judiciales['RUC_DNI'].nunique()

(26, 26)

In [88]:
conformacion_juridica = pd.read_csv('../Datos/OSCE/conformacion_juridica.csv', sep='|', encoding='latin-1', dtype='str')

In [91]:
conformacion_juridica.head()

,FECHA_CORTE,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,NOMBRE_RAZONODENOMINACIONSOCIAL,RUC,TIPO_CONF_JURIDICA,FECHA_INICIO_VIGENCIA,ID_FORMA_SOCIETARIA,DE_FORMA_SOCIETARIA
0,20220401,CARNET DE EXTRANJERIA,\t18077465,VILLACORTA VILCA AMABLE TEMPORA,20605583955,SOCIO_ACCIONISTA_TITULAR,20200109,3,EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA
1,20220401,CARNET DE EXTRANJERIA,\t2012414897,EAFC MAQUISISTEMA S.A.,20600053273,SOCIO_ACCIONISTA_TITULAR,20180407,1,SOCIEDAD ANONIMA CERRADA
2,20220401,CARNET DE EXTRANJERIA,\t2055287805,PANAMERICAN INVESTMENTS HOLDINGS S.A.C.,20100017149,SOCIO_ACCIONISTA_TITULAR,20160901,5,SOCIEDAD ANONIMA
3,20220401,CARNET DE EXTRANJERIA,\t32987431,CECILIA RAQUEL MOSTACERO OLIDEN,20603494581,SOCIO_ACCIONISTA_TITULAR,20181006,3,EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA
4,20220401,CARNET DE EXTRANJERIA,\t72128966,MARIA MILAGROS JIMENEZ TAQUIRE,20600488059,SOCIO_ACCIONISTA_TITULAR,20180405,1,SOCIEDAD ANONIMA CERRADA


In [99]:
conformacion_juridica.shape[0], conformacion_juridica['RUC'].nunique(), conformacion_juridica['NUMERO_DOCUMENTO'].nunique()

(953142, 228634, 458477)

In [92]:
sancionados = pd.read_csv('../Datos/OSCE/sancionados.csv', sep='|', encoding='latin-1', dtype='str')

In [93]:
sancionados.head()

,FECHA_CORTE,RUC,NOMBRE_RAZONODENOMINACIONSOCIAL,FECHA_INICIO,FECHA_FIN,NUMERO_RESOLUCION,ID_MOTIVO_INFRACCION,DE_MOTIVO_INFRACCION
0,20220430,20100994128,CONSTRUCTORA DOS DE MAYO S.A.,19980806,NaN,074-1998-TL,12,RESCISION ADMINISTRATIVA DEL CONTRATO
1,20220430,20100247063,CEFOISA,19990218,NaN,008-1999-TC-S1,12,RESCISION ADMINISTRATIVA DEL CONTRATO
2,20220430,10092982349,VERA GUTIERREZ RODOLFO DAVID - INGENIERO,19961021,NaN,161-1996-TL,12,RESCISION ADMINISTRATIVA DEL CONTRATO
3,20220430,20113374650,SANTA GRACIELA S.A. CONTRATISTAS GENERALES,19951001,NaN,116-1995-TL,12,RESCISION ADMINISTRATIVA DEL CONTRATO
4,20220430,20124260460,NELLO PERALTA CONTRATISTAS GENERALES S.R.L.,19980123,NaN,193-1997-TL,12,RESCISION ADMINISTRATIVA DEL CONTRATO


In [110]:
sancionados_agg = sancionados.groupby('RUC').agg({
                                            'NOMBRE_RAZONODENOMINACIONSOCIAL': 'max',
                                            'RUC': 'count',
                                            'FECHA_INICIO': ['min', 'max'],
                                            'DE_MOTIVO_INFRACCION': 'max'
                                            }).reset_index()

In [111]:
sancionados_agg.columns = ['RUC', 'NOMBRE_RAZONODENOMINACIONSOCIAL', 'cantSanciones', 'fechaSancionMin', 'fechaSancionMax', 'motivoSancion']

In [118]:
sancionados_multa = pd.read_csv('../Datos/OSCE/sancionados_multa.csv', sep='|', encoding='latin-1', dtype='str')
sancionados_multa['MONTO'] = sancionados_multa['MONTO'].apply(lambda x: float(x))

In [119]:
sancionados_multa.tail()

,FECHA_CORTE,RUC,NOMBRE_RAZONODENOMINACIONSOCIAL,FECHA_INICIO,FECHA_FIN,NUMERO_RESOLUCION,ID_MOTIVO_INFRACCION,DE_MOTIVO_INFRACCION,MONTO
2130,20220430,99000019701,CONSTRUCCIONES CIVILES Y TOPOGRAFIA SAS CONCIT...,20190918,20200518,2445-2019-TCE-S3,"207,",Incumplir con su obligación de perfeccionar el...,2831592.72
2131,20220430,99000019701,CONSTRUCCIONES CIVILES Y TOPOGRAFIA SAS CONCIT...,20220111,20221011,4341-2021-TCE-S1,"237,",b) Incumplir injustificadamente con su obligac...,1790156.67
2132,20220430,99000020351,SEMPSA JOYERIA PLATERIA S.A.,20171124,20180724,2428-2017-TCE-S4,"196,",Incumplir injustificadamente con su obligación...,37165.35
2133,20220430,99000027738,NESE PARTNERS S.A.,20191120,20200420,2927-2019-TCE-S1,"223,",b) Incumplir injustificadamente con su obligac...,24333.33
2134,20220430,99000030208,CMEC ENGINEERING MACHINERY IMPORT & EXPORT CO....,20220406,20221006,915-2022-TCE-S1,"237,",b) Incumplir injustificadamente con su obligac...,6245632.32


In [121]:
multa_agg = sancionados_multa.groupby('RUC').agg({
                                            'NOMBRE_RAZONODENOMINACIONSOCIAL': 'max',
                                            'RUC': 'count',
                                            'FECHA_INICIO': ['min', 'max'],
                                            'DE_MOTIVO_INFRACCION': 'max',
                                            'MONTO': 'sum'
                                            }).reset_index()

In [122]:
multa_agg.columns = ['RUC', 'NOMBRE_RAZONODENOMINACIONSOCIAL', 'cantMultas', 'fechaMultaMin', 'fechaMultaMax', 'motivoMulta', 'montoMulta']

In [123]:
multa_agg.head()

,RUC,NOMBRE_RAZONODENOMINACIONSOCIAL,cantMultas,fechaMultaMin,fechaMultaMax,motivoMulta,montoMulta
0,10000345739,FLORES GARCIA LUIS ARTIDORO,1,20210803,20210803,b) Incumplir injustificadamente con su obligac...,19253.75
1,10000818025,PANDURO CHANCHARI ESLEIDER,1,20211103,20211103,b) Incumplir injustificadamente con su obligac...,22000.00
2,10002186719,RUIZ DIOSES FLAVIO,3,20210810,20220418,b) Incumplir injustificadamente con su obligac...,73066.66
3,10002286969,CRUZ MEDINA ARELI,1,20191230,20191230,a) Desistirse o retirar injustificadamente su ...,18079.00
4,10002397248,GARRIDO LOPEZ CARLOS MANUEL,1,20180711,20180711,Incumplir injustificadamente con su obligación...,7053.80


In [142]:
#penalidades = pd.read_csv('../Datos/OSCE/penalidades.csv', sep='|', encoding='latin-1', dtype='str')
penalidades = pd.read_excel('../Datos/OSCE/penalidades.csv', header=0, dtype='str')

penalidades['ID CONTRATO'] = penalidades['ID CONTRATO'].apply(lambda x: int(float(x)))
penalidades['MONTO'] = penalidades['MONTO'].apply(lambda x: float(x))

In [143]:
penalidades.head()

,ID CONTRATO,RUC CONTRATISTA,TIPO PENALIDAD,OBJETO CONTRATO,ENTIDAD CONTRATANTE,FECHA PENALIDAD,DESCRIPCION/MOTIVO,MONTO
0,2050788,20450384322,Por Mora,Bien,GOBIERNO REGIONAL CAJAMARCA SALUD SAN IGNACIO,25/05/21,POR CONCEPTO DE PENALIDAD APLICADA AL PROVEEDO...,15012.00
1,2016253,10215388196,Otra Penalidad,Servicio,UNIVERSIDAD NACIONAL CIRO ALEGRIA,07/07/21,POR NO PRESENTAR HABILIDAD DE LOS PROFESIONALE...,7700.00
2,2016248,20605620541,Por Mora,Servicio,UNIVERSIDAD NACIONAL CIRO ALEGRIA,18/06/21,MORA POR UN DIA DE RETRASO EN LA PRESENTACION ...,611.11
3,2016253,10215388196,Por Mora,Servicio,UNIVERSIDAD NACIONAL CIRO ALEGRIA,07/07/21,POR UN DIA DE MORA EN EL TERCER ENTREGABLE,613.33
4,2092744,20565412257,Otra Penalidad,Servicio,PROGRAMA NACIONAL DE CENTROS JUVENILES,24/02/22,PENALIDADES VARIOS,7655.00


In [146]:
penalidades_agg = penalidades.groupby(['ID CONTRATO', 'RUC CONTRATISTA']).agg(
                                                    {
                                                        'ID CONTRATO': 'count',
                                                        'FECHA PENALIDAD': ['min', 'max'],
                                                        'TIPO PENALIDAD': 'max',
                                                        'DESCRIPCION/MOTIVO': 'max',
                                                        'MONTO': 'sum'
                                                    }
                                                    ).reset_index()

In [149]:
penalidades_agg.columns = ['ID CONTRATO', 'RUC CONTRATISTA', 'cantPenalidades', 'fechaPenalidadMin', 'fechaPenalidadMax', 'tipoPenalidad', 'motivoPenalidad', 'montoPenalidad']

In [150]:
penalidades_agg.shape[0], penalidades_agg['ID CONTRATO'].nunique()

(698, 698)

In [151]:
penalidades_agg.head()

,ID CONTRATO,RUC CONTRATISTA,cantPenalidades,fechaPenalidadMin,fechaPenalidadMax,tipoPenalidad,motivoPenalidad,montoPenalidad
0,2000043,20601218250,1,14/12/20,14/12/20,Otra Penalidad,Otras penalidades,430.00
1,2000053,20359742615,6,09/04/21,29/01/21,Otra Penalidad,Puestos de seguridad y vigilancia sin cubrir c...,20735.00
2,2000144,20450372669,1,28/07/21,28/07/21,Otra Penalidad,Otras penalidades al Servicio de Alimentacion ...,4162.40
3,2000378,20490234137,10,02/02/21,25/04/21,Por Mora,penalidad por mora - correspondiente al mes de...,33602.39
4,2000552,10434381288,1,05/10/20,05/10/20,Por Mora,PENALIDAD POR MORA EN LA ENTREGA DEL PLAN DE T...,5850.00


In [152]:
penalidades2018 = pd.read_excel('../Datos/OSCE/penalidades2018.csv', header=0, dtype='str')
penalidades2018['ID CONTRATO'] = penalidades2018['ID CONTRATO'].apply(lambda x: int(float(x)))
penalidades2018['MONTO'] = penalidades2018['MONTO'].apply(lambda x: float(x))

In [154]:
penalidades2018_agg = penalidades2018.groupby(['ID CONTRATO', 'RUC CONTRATISTA']).agg(
                                                    {
                                                        'ID CONTRATO': 'count',
                                                        'FECHA PENALIDAD': ['min', 'max'],
                                                        'TIPO PENALIDAD': 'max',
                                                        'DESCRIPCION/MOTIVO': 'max',
                                                        'MONTO': 'sum'
                                                    }
                                                    ).reset_index()

In [155]:
penalidades2018_agg.columns = ['ID CONTRATO', 'RUC CONTRATISTA', 'cantPenalidades', 'fechaPenalidadMin', 'fechaPenalidadMax', 'tipoPenalidad', 'motivoPenalidad', 'montoPenalidad']

In [156]:
penalidades2018_agg.head()

,ID CONTRATO,RUC CONTRATISTA,cantPenalidades,fechaPenalidadMin,fechaPenalidadMax,tipoPenalidad,motivoPenalidad,montoPenalidad
0,558873,C0002042265,2,10/10/13,16/06/14,POR MORA,retraso en la ejecución,306890.43
1,563095,C0002069105,2,15/03/18,15/03/18,POR MORA,PENALIDAD NO. SJM180020 - POR INCUMPLIMIENTO D...,18144.80
2,578956,20100083362,1,16/10/13,16/10/13,OTRA PENALIDAD,OTRAS PENALIDADES,1297.13
3,592091,10410713506,1,27/03/21,27/03/21,POR MORA,PENALIDAD MAXIMA,48131.63
4,638204,C0002227330,1,23/06/16,23/06/16,OTRA PENALIDAD,UP TIME,1253.22


In [217]:
penalidades2018_agg.columns = ['idContrato', 'RUC CONTRATISTA', 'cantPenalidades', 'fechaPenalidadMin', 'fechaPenalidadMax', 'tipoPenalidad',
       'motivoPenalidad', 'montoPenalidad']

In [219]:
penalidades2018_agg.shape[0], penalidades2018_agg['idContrato'].nunique()

(2022, 2022)

In [220]:
Resultados_Contratos_f_2 = pd.merge(Resultados_Contratos_f, penalidades2018_agg.drop('RUC CONTRATISTA', axis=1), on='idContrato', how='left')

In [221]:
Resultados_Contratos_f_2.shape[0], Resultados_Contratos_f_2['idContrato'].nunique()

(85725, 85725)

In [222]:
Resultados_Contratos_f_2.drop(['CUI'], axis=1).to_csv('../Datos/Matrices/Matriz_contratos.txt', sep='\t', index=False)

#### Unir sanciones

In [183]:
inhabilitaciones_judiciales.columns = ['FECHA_CORTE', 'RUC', 'NOMBRE_RAZONODENOMINACIONSOCIAL',
       'ORGANO_JURISDICCIONAL', 'NUMERO_RESOLUCION', 'FECHA_INICIO_INHABILITACION', 'FECHA_FIN_INHABILITACION']

In [190]:
inhabilitaciones_judiciales['tieneINHABILITACION'] = 1

In [184]:
listado_id = ['RUC', 'NOMBRE_RAZONODENOMINACIONSOCIAL']

In [199]:
dfSanciones = pd.concat([inhabilitaciones_judiciales[listado_id], sancionados_agg[listado_id], multa_agg[listado_id]]).drop_duplicates().reset_index()

In [200]:
dfSanciones['RN'] = dfSanciones.sort_values(['NOMBRE_RAZONODENOMINACIONSOCIAL'], ascending=[False]) \
             .groupby(['RUC']) \
             .cumcount() + 1

dfSanciones = dfSanciones[dfSanciones['RN'] == 1].drop(['index', 'RN'], axis=1).copy()

In [201]:
dfSanciones = pd.merge(dfSanciones, inhabilitaciones_judiciales[['RUC', 'FECHA_INICIO_INHABILITACION', 'FECHA_FIN_INHABILITACION', 'tieneINHABILITACION']], on='RUC', how='left')
dfSanciones = pd.merge(dfSanciones, sancionados_agg[['RUC', 'cantSanciones', 'fechaSancionMin', 'fechaSancionMax', 'motivoSancion']], on='RUC', how='left')
dfSanciones = pd.merge(dfSanciones, multa_agg[['RUC', 'cantMultas', 'fechaMultaMin', 'fechaMultaMax', 'motivoMulta', 'montoMulta']], on='RUC', how='left')

In [202]:
dfSanciones.head()

,RUC,NOMBRE_RAZONODENOMINACIONSOCIAL,FECHA_INICIO_INHABILITACION,FECHA_FIN_INHABILITACION,tieneINHABILITACION,cantSanciones,fechaSancionMin,fechaSancionMax,motivoSancion,cantMultas,fechaMultaMin,fechaMultaMax,motivoMulta,montoMulta
0,10011590921,MURGA MONTOYA JOSE ELIAS,20180606,20190606,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10015569943,MAMANI CHURA JORGE SOLON,20181017,20191017,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10040039711,BARRETO MARCELO TEODORO,20170428,20250428,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10086869093,ARCE JIMENEZ ERNESTO FROY,20190502,20191102,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10107797691,LARA MALCA JUAN OSWALDO,20180703,20201103,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
dfSanciones.to_csv('../Datos/Matrices/Matriz_proveedores_sancionados.txt', sep='\t', index=False)

In [203]:
dfSanciones.shape[0], dfSanciones['RUC'].nunique()

(7465, 7465)

In [204]:
dfSanciones['cantSanciones'].value_counts()

1.0     4750
2.0      909
3.0      252
4.0      106
5.0       38
6.0       25
8.0       14
7.0       11
9.0        9
10.0       5
14.0       2
11.0       2
12.0       1
15.0       1
17.0       1
35.0       1
Name: cantSanciones, dtype: int64

### Unir contratos

In [115]:
def conv_float(x):
    try:
        return float(x)
    except:
        return None

In [116]:
Resultados_Contratos = pd.read_csv('../Datos/Matrices/Matriz_contratos.txt', sep='\t')

In [117]:
Resultados_Contratos['montoContrato'] = Resultados_Contratos['montoContrato'].apply(lambda x: conv_float(x))

In [118]:
Resultados_Contratos.head()

,idContrato,entidadContratante,contratista,descripcionContrato,numeroContrato,rucCodigoContratista,tipoMoneda,montoContrato,fechaSuscripcionContrato,fechaInicioVigenciaContrato,fechaPublicacionContrato,fechaFinVigenciaContrato,lugar,destinatarioPago,estadoSituacional,archivoAdjunto,entidadConvocante,rucDestinoPago,nomenclaturaProceso,montoPresupuesto,montoCCP,montoPrevision,archivoAdjunto2,idDocumento2,tamanioDocumento2,fechaDocumento2,indPresupuesto,listaItemsContrato,listaGarantiaContrato,listaAccionContrato,listaProyectoContrato,listaResolucionDisputas,listaConciliaciones,listaArbitrajes,listaPresupuestoGeneral,idDocumento,tamanioDocumento,fechaDocumento,nIndConsorcio,idDocumentoConsorcio,arbitraje,conciliacion,garantia,resolucionDisputas,cantPenalidades,fechaPenalidadMin,fechaPenalidadMax,tipoPenalidad,motivoPenalidad,montoPenalidad
0,2095549,20192245461-MUNICIPALIDAD DISTRITAL DE LA COIPA,20570706170-CONSORCIO SUPERVISOR LOMA LARGA,SERVICIO DE CONSULTORÍA DE OBRA PARA LA SUPERV...,CONTRATO CONCURSO PUBLICO N° 001-2021-MDLC/ABAST,20570706170,Soles,488850.00,30/12/2021,10/01/2022,09/04/2022,04/02/2023,CAJAMARCA,20608853651-CONSORCIO SUPERVISOR LOMA LARGA,Publicado,contrato super loma larga.pdf,20192245461-MUNICIPALIDAD DISTRITAL DE LA COIPA,20608853651,CP-SM-2-2021-MDLC/CS-1,488850,488850,524421.16,CONT CONSORCIO SUPER LOMA LARGA.pdf,389798,1198573,05/01/2022 12:34 PM,1,[{'idItem': 201477 'nroItem': '1' 'cubso': '81...,[],[],[{'codigoUnicoInversion': '2337550' 'descripci...,[],[],[],NaN,389797,1630133,05/01/2022 12:34 PM,1.0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2108677,20153408191-MARINA DE GUERRA DEL PERU,20509585661-ADR TECNOLOGY S.A.C.,ADQUISICION REPUESTOS PARA ESTACIONES AUTOMATI...,003-2022,20509585661,Soles,97790.00,06/04/2022,07/04/2022,09/04/2022,20/07/2022,CALLAO,20509585661-ADR TECNOLOGY S.A.C.,Publicado,CONTRATO AS 03.pdf,20153408191-MARINA DE GUERRA DEL PERU,20509585661,AS-SM-3-2022-MGP/DIHIDRONAV-1,101220,101220,,,,,,1,[{'idItem': 215722 'nroItem': '1' 'cubso': '-'...,[],[],[],[],[],[],NaN,433964,730100,09/04/2022 07:15 PM,0.0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2108676,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,CONTRATACION DE SUMINISTRO DE COMBUSTIBLE (DIE...,CONTRATO N° 065-2022-MPM,20560151757,Soles,214700.00,23/03/2022,24/03/2022,09/04/2022,30/09/2022,HUANUCO,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,Publicado,CONTRATO N° 065-2022-MPM.pdf,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757,SIE-SIE-3-2022-MPM/CS-1,214700,214700,,,,,,1,[{'idItem': 215721 'nroItem': '1' 'cubso': '15...,[],[],[{'codigoUnicoInversion': '2526872' 'descripci...,[],[],[],NaN,433963,2333761,09/04/2022 12:26 PM,0.0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2108675,20383430250-POLICÍA NACIONAL DEL PERÚ - JEFATU...,20513345748-GERAL SERVICIOS E.I.R.L.,ALIMENTO BALANCEADO HIPOALERGÉNICO PARA CANES ...,02-2022-VII DIRTEPOL LIMA,20513345748,Soles,53406.72,09/04/2022,11/04/2022,09/04/2022,15/04/2022,LIMA,20513345748-GERAL SERVICIOS E.I.R.L.,Publicado,CONTRATO 02-2022-VII DIRTEPOL LIMA.pdf,20383430250-POLICÍA NACIONAL DEL PERÚ - JEFATU...,20513345748,COMPRE-SM-1-2022-VII DIRTEPOL L-1,53406.72,53406.72,,,,,,1,[{'idItem': 215720 'nroItem': '1' 'cubso': '10...,[],[],[],[],[],[],NaN,433962,1531229,09/04/2022 12:23 PM,0.0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2108674,20546303951-CORTE SUPERIOR DE JUSTICIA DE LIMA,20514441163-CONSTRUCTORA EL ROBLE S.A.C.,Servicio de arrendamiento de un inmueble para ...,Contrato N° 012-2022-s-p-csjli-pj,20514441163,Soles,10622808.36,08/04/2022,09/04/2022,09/04/2022,07/06/2025,LIMA,20514441163-CONSTRUCTORA EL ROBLE S.A.C.,Publicado,CONTRATO N° 12-2022 arrendamiento materia civi...,20546303951-CORTE SUPERIOR DE JUSTICIA DE LIMA,20514441163,DIRECTA-PROC-3-2022-OEC-CSJLI/PJ-1,21245616.72,3662155.58,17583461.14,,,,,1,[{'idItem': 215719 'nroItem': '1' 'cubso': '80...,[{'idGarantia': 30203 'claseGarantia': 'Garant...,[],[],[],[],[],NaN,433956,85

In [129]:
Resultados_Contratos[['idContrato']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85725 entries, 0 to 85724
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   idContrato  85725 non-null  int64
dtypes: int64(1)
memory usage: 669.9 KB


In [93]:
Resultados_Contratos.shape[0], Resultados_Contratos['idContrato'].nunique()

(85725, 85725)

In [110]:
df_cui = pd.read_csv('../Datos/Matrices/Matriz_relacion_CUI_contratos.txt', sep='\t').drop('descripcionProyecto', axis=1).drop_duplicates()

In [130]:
df_cui_2 = pd.merge(df_cui, Resultados_Contratos[['idContrato', 'montoContrato']], how='left', on='idContrato')

In [131]:
df_cui_2['RN'] = df_cui_2.sort_values(['montoContrato',], ascending=[False]) \
             .groupby(['codigoUnicoInversion']) \
             .cumcount() + 1

df_cui_2 = df_cui_2[df_cui_2['RN']==1].drop(['RN', 'montoContrato'], axis=1).copy()

In [132]:
df_cui_2.columns = ['CODIGO_UNICO', 'idContrato']

In [134]:
df_cui_2 = df_cui_2[df_cui_2['idContrato'].fillna(-1)>0].copy()

In [135]:
df_cui_2['idContrato'] = df_cui_2['idContrato'].apply(lambda x: int(x))

In [136]:
df_cui_2.head()

,CODIGO_UNICO,idContrato
1,2526872,2108676
2,2503696,2108673
3,2540634,2107617
4,2495830,2108667
5,2061710,2108664


In [137]:
dfContratosCui = pd.merge(df_cui_2, Resultados_Contratos, how='left', on='idContrato')

In [138]:
dfContratosCui.head()

,CODIGO_UNICO,idContrato,entidadContratante,contratista,descripcionContrato,numeroContrato,rucCodigoContratista,tipoMoneda,montoContrato,fechaSuscripcionContrato,fechaInicioVigenciaContrato,fechaPublicacionContrato,fechaFinVigenciaContrato,lugar,destinatarioPago,estadoSituacional,archivoAdjunto,entidadConvocante,rucDestinoPago,nomenclaturaProceso,montoPresupuesto,montoCCP,montoPrevision,archivoAdjunto2,idDocumento2,tamanioDocumento2,fechaDocumento2,indPresupuesto,listaItemsContrato,listaGarantiaContrato,listaAccionContrato,listaProyectoContrato,listaResolucionDisputas,listaConciliaciones,listaArbitrajes,listaPresupuestoGeneral,idDocumento,tamanioDocumento,fechaDocumento,nIndConsorcio,idDocumentoConsorcio,arbitraje,conciliacion,garantia,resolucionDisputas,cantPenalidades,fechaPenalidadMin,fechaPenalidadMax,tipoPenalidad,motivoPenalidad,montoPenalidad
0,2526872,2108676,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,CONTRATACION DE SUMINISTRO DE COMBUSTIBLE (DIE...,CONTRATO N° 065-2022-MPM,20560151757,Soles,214700.00,23/03/2022,24/03/2022,09/04/2022,30/09/2022,HUANUCO,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,Publicado,CONTRATO N° 065-2022-MPM.pdf,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757,SIE-SIE-3-2022-MPM/CS-1,214700,214700,,,,,,1,[{'idItem': 215721 'nroItem': '1' 'cubso': '15...,[],[],[{'codigoUnicoInversion': '2526872' 'descripci...,[],[],[],NaN,433963,2333761,09/04/2022 12:26 PM,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2503696,2108673,20193995242-MUNICIPALIDAD DISTRITAL DE YAUCA D...,20494409951-CORPORATION SAGESSE S.A.C.,"CONTRATO PARA LA EJECUCION DE LA OBRA: ""MEJORA...",S/N,20494409951,Soles,350444.71,04/04/2022,05/04/2022,08/04/2022,04/07/2022,ICA,20494409951-CORPORATION SAGESSE S.A.C.,Publicado,CONTRATO PAMPAHUASI.pdf,20193995242-MUNICIPALIDAD DISTRITAL DE YAUCA D...,20494409951,AS-SM-1-2022-CS-MDYR-1,350444.71,350444.71,,,,,,1,[{'idItem': 215718 'nroItem': '1' 'cubso': '72...,[],[],[{'codigoUnicoInversion': '2503696' 'descripci...,[],[],[],NaN,433955,3339051,08/04/2022 11:43 PM,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2540634,2107617,20143157301-MUNICIPALIDAD DISTRITAL DE AYAHUANCO,20568721676-TRANSPORTES MARFEM S.A.C.,ADQUISICIÓN DE COMBUSTIBLE DIESEL B5 PARA LA E...,39,20568721676,Soles,106200.00,24/03/2022,24/03/2022,08/04/2022,06/05/2022,AYACUCHO,20568721676-TRANSPORTES MARFEM S.A.C.,Publicado,c2.pdf,20143157301-MUNICIPALIDAD DISTRITAL DE AYAHUANCO,20568721676,SIE-SIE-2-2022-MDA/OEC-1,318600,318600,,,,,,1,[{'idItem': 214566 'nroItem': '1' 'cubso': '15...,[],[],[{'codigoUnicoInversion': '2540634' 'descripci...,[],[],[],NaN,431767,3082007,04/04/2022 01:14 PM,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2495830,2108667,20189021764-MUNICIPALIDAD DISTRITAL DE HUACAR,10459749760-CONSORCIO VIAL HUACAR,SERVICIO DE CONSULTORIA DE OBRA PARA LA ELABOR...,001-2022-MDH-A,10459749760,Soles,100000.00,08/04/2022,11/04/2022,08/04/2022,11/07/2022,HUANUCO,10459749760-DUEÑAS SANTILLAN JOSE DAIMBLER,Publicado,contrato vial huacar.pdf,20189021764-MUNICIPALIDAD DISTRITAL DE HUACAR,10459749760,AS-SM-1-2022-MDH/CS-1,100000,100000,,CONTRATO DE CONSORCIO vial huacar.pdf,433942,5804670,08/04/2022 07:51 PM,1,[{'idItem': 215708 'nroItem': '1' 'cubso': '81...,[],[],[{'codigoUnicoInversion': '2495830' 'descripci...,[],[],[],NaN,433941,2183435,08/04/2022 07:51 PM,1.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2061710,2108664,20486021692-GOBIERNO REGIONAL DE JUNIN SEDE CE...,20602262201-CONSORCIO SAN FELIPE,CONTRATACIÓN DE LA EJECUCIÓN DE LA OBRA: CONST...,30-2022-GRJ/ORAF,20602262201,Soles,3423667.13,05/04/2022,06/04/2022,08/04/2022,17/09/2022,JUNIN,20574732400-CONSTRUCTORA E INMOBILIARIOS SAN F...,Publicado,contrato 30-2022.pdf,20486021692-GOBIERNO REGIONAL DE JUNIN SEDE CE...,20574732400,AS-SM-66-2021-GRJ-CS-2,3458249.64,3458249.64,,contrato de consorcio.pdf,433934,1574316,08/04/2022 07:33 PM,1,[{'idItem': 215704 'nroItem': '1' 'cubso': '7

In [142]:
dfContratosCui[['cantPenalidades', 'tipoPenalidad', 'motivoPenalidad', 'montoPenalidad']] = dfContratosCui[['cantPenalidades', 'tipoPenalidad', 'motivoPenalidad', 'montoPenalidad']].fillna(0)

In [143]:
dfSanciones = pd.read_csv('../Datos/Matrices/Matriz_proveedores_sancionados.txt', sep='\t')

In [145]:
dfSanciones.shape[0], dfSanciones['RUC'].nunique()

(7465, 7465)

In [146]:
dfContratosCui.rename(columns = {'rucCodigoContratista':'RUC'}, inplace = True)

In [150]:
dfSanciones['RUC'] = dfSanciones['RUC'].apply(lambda x: str(x))

In [151]:
dfContratosCui_2 = pd.merge(dfContratosCui, dfSanciones, how='left', on='RUC')

In [152]:
dfContratosCui_2.head()

,CODIGO_UNICO,idContrato,entidadContratante,contratista,descripcionContrato,numeroContrato,RUC,tipoMoneda,montoContrato,fechaSuscripcionContrato,fechaInicioVigenciaContrato,fechaPublicacionContrato,fechaFinVigenciaContrato,lugar,destinatarioPago,estadoSituacional,archivoAdjunto,entidadConvocante,rucDestinoPago,nomenclaturaProceso,montoPresupuesto,montoCCP,montoPrevision,archivoAdjunto2,idDocumento2,tamanioDocumento2,fechaDocumento2,indPresupuesto,listaItemsContrato,listaGarantiaContrato,listaAccionContrato,listaProyectoContrato,listaResolucionDisputas,listaConciliaciones,listaArbitrajes,listaPresupuestoGeneral,idDocumento,tamanioDocumento,fechaDocumento,nIndConsorcio,idDocumentoConsorcio,arbitraje,conciliacion,garantia,resolucionDisputas,cantPenalidades,fechaPenalidadMin,fechaPenalidadMax,tipoPenalidad,motivoPenalidad,montoPenalidad,NOMBRE_RAZONODENOMINACIONSOCIAL,FECHA_INICIO_INHABILITACION,FECHA_FIN_INHABILITACION,tieneINHABILITACION,cantSanciones,fechaSancionMin,fechaSancionMax,motivoSancion,cantMultas,fechaMultaMin,fechaMultaMax,motivoMulta,montoMulta
0,2526872,2108676,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,CONTRATACION DE SUMINISTRO DE COMBUSTIBLE (DIE...,CONTRATO N° 065-2022-MPM,20560151757,Soles,214700.00,23/03/2022,24/03/2022,09/04/2022,30/09/2022,HUANUCO,20560151757-INVERSIONES Y SERVICIOS WACRACHUCO...,Publicado,CONTRATO N° 065-2022-MPM.pdf,20228453146-MUNICIPALIDAD PROVINCIAL DE MARAÑO...,20560151757,SIE-SIE-3-2022-MPM/CS-1,214700,214700,,,,,,1,[{'idItem': 215721 'nroItem': '1' 'cubso': '15...,[],[],[{'codigoUnicoInversion': '2526872' 'descripci...,[],[],[],NaN,433963,2333761,09/04/2022 12:26 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2503696,2108673,20193995242-MUNICIPALIDAD DISTRITAL DE YAUCA D...,20494409951-CORPORATION SAGESSE S.A.C.,"CONTRATO PARA LA EJECUCION DE LA OBRA: ""MEJORA...",S/N,20494409951,Soles,350444.71,04/04/2022,05/04/2022,08/04/2022,04/07/2022,ICA,20494409951-CORPORATION SAGESSE S.A.C.,Publicado,CONTRATO PAMPAHUASI.pdf,20193995242-MUNICIPALIDAD DISTRITAL DE YAUCA D...,20494409951,AS-SM-1-2022-CS-MDYR-1,350444.71,350444.71,,,,,,1,[{'idItem': 215718 'nroItem': '1' 'cubso': '72...,[],[],[{'codigoUnicoInversion': '2503696' 'descripci...,[],[],[],NaN,433955,3339051,08/04/2022 11:43 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2540634,2107617,20143157301-MUNICIPALIDAD DISTRITAL DE AYAHUANCO,20568721676-TRANSPORTES MARFEM S.A.C.,ADQUISICIÓN DE COMBUSTIBLE DIESEL B5 PARA LA E...,39,20568721676,Soles,106200.00,24/03/2022,24/03/2022,08/04/2022,06/05/2022,AYACUCHO,20568721676-TRANSPORTES MARFEM S.A.C.,Publicado,c2.pdf,20143157301-MUNICIPALIDAD DISTRITAL DE AYAHUANCO,20568721676,SIE-SIE-2-2022-MDA/OEC-1,318600,318600,,,,,,1,[{'idItem': 214566 'nroItem': '1' 'cubso': '15...,[],[],[{'codigoUnicoInversion': '2540634' 'descripci...,[],[],[],NaN,431767,3082007,04/04/2022 01:14 PM,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2495830,2108667,20189021764-MUNICIPALIDAD DISTRITAL DE HUACAR,10459749760-CONSORCIO VIAL HUACAR,SERVICIO DE CONSULTORIA DE OBRA PARA LA ELABOR...,001-2022-MDH-A,10459749760,Soles,100000.00,08/04/2022,11/04/2022,08/04/2022,11/07/2022,HUANUCO,10459749760-DUEÑAS SANTILLAN JOSE DAIMBLER,Publicado,contrato vial huacar.pdf,20189021764-MUNICIPALIDAD DISTRITAL DE HUACAR,10459749760,AS-SM-1-2022-MDH/CS-1,100000,100000,,CONTRATO DE CONSORCIO vial huacar.pdf,433942,5804670,08/04/2022 07:51 PM,1,[{'idItem': 215708 'nroItem': '1' 'cubso': '81...,[],[],[{'codigoUnicoInversion': '2495830' 'descripci...,[],[],[],NaN,433941,2183435,08/04/2022 07:51 PM,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2061710,2108664,20486021692-GOBIERNO REGIONAL DE JUNIN SEDE CE...,20602262201-CONSORCIO SAN FELIPE,CONTRATACIÓN DE LA EJECUC

In [153]:
columnas_sanciones = ['tieneINHABILITACION', 'cantSanciones', 'motivoSancion', 'cantMultas', 'montoMulta']

dfContratosCui_2[columnas_sanciones] = dfContratosCui_2[columnas_sanciones].fillna(0)

In [ ]:
dfContratosCui_2

In [155]:
dfContratosCui_2.to_csv('../Datos/Matrices/Matriz_proveedores_consolidado.txt', sep='\t', index=False)

In [301]:
def conv_int(x):
    try:
        return int(x)
    except:
        return None

In [345]:
dfInfobra['CODIGO_UNICO'] = dfInfobra['CODIGO_UNICO'].astype(int)
dfSSI['CODIGO_UNICO'] = dfSSI['CODIGO_UNICO'].apply(lambda x: conv_int(x))
dfUbicacion['CODIGO_UNICO'] = dfUbicacion['CODIGO_UNICO'].astype(int)
dfModificaciones_agg['CODIGO_UNICO'] = dfModificaciones_agg['CODIGO_UNICO'].astype(int)

In [346]:
dfSSI['RN'] = dfSSI.sort_values(['COSTO_ACTUALIZADO',], ascending=[False]) \
             .groupby(['CODIGO_UNICO']) \
             .cumcount() + 1

dfSSI = dfSSI[dfSSI['RN']==1].drop(['RN'], axis=1).copy()

In [352]:
dfSSI = dfSSI[~dfSSI['CODIGO_UNICO'].isna()].copy()

In [353]:
dfSSI['CODIGO_UNICO'] = dfSSI['CODIGO_UNICO'].apply(lambda x: int(x))

In [358]:
datos = pd.merge(dfInfobra, dfSSI, how='left', on='CODIGO_UNICO')
datos = pd.merge(datos, dfUbicacion, how='left', on='CODIGO_UNICO')
datos = pd.merge(datos, dfAvance_agg, how='left', on='CODIGO_UNICO')
datos = pd.merge(datos, dfModificaciones_agg, how='left', on='CODIGO_UNICO')

In [359]:
datos['NOBR_ID'] = datos['NOBR_ID'].astype(int)

In [363]:
datos.to_csv('../Datos/Matrices/Matriz_infobra.txt', sep='|', index=False)

In [23]:
datos = pd.read_csv('../Datos/Matrices/Matriz_infobra.txt', sep='|')

In [24]:
datos.head()

,NOBR_ID,COBR_DESCRI,CODCONSUCODE,COBR_DIRECCION,NOBR_TIPEJE,COPA_DESCRI,RUBRO,CODCONSUCODE_DESCRI,COBR_NOMBRERESIDENTE,COBR_DNIRESIDENTE,DOBR_FEC_INI_OBRA,NOBR_CODSNIP,DSNP_FECVERI,NSNP_MONTOVERI,DSNP_FECAPR,NSNP_MONTOAPR,CSNP_CODSIAF,CSNP_DESCRI,CODIGO_UNICO,FECHA_ACTUALIZACION,FECHA_INICIO,UNIDAD_OPMI,SECTOR,ENTIDAD,FUNCION,PIM_ANO_VIGENTE,DEV_ANO_VIGENTE,DEV_ACUMULADO,NIVEL,COD_UNIDAD_UEI,DES_UNIDAD_UEI,DES_UNIDAD_UF,COSTO_ACTUALIZADO,MTO_VIABLE,NOMBRE_INVERSION,FEC_REGISTRO,FEC_VIABLE,TIPO_FORMATO,MES_ANO_PRI_DEV,MES_ANO_ULT_DEV,CERTIFICADO_ANO_ACTUAL,GIRADO_ANO_ACTUAL,COMPROMETIDO_ANO_ACTUAL,MTO_PMI_1,MTO_PMI_2,MTO_PMI_3,MTO_PMI_4,MTO_CartFza,NOMBRE_OPMI,ESTADO,FECHA_ET,CIERRE_REGISTRADO,FEC_INI_EJ,FEC_FIN_EJ,MODAL_EJEC,MARCO,COD_SNIP,IND_REG_PMI,DES_PROGRAMA,DES_SUB_PROGRAMA,SITUACION,BENEFICIARIO,COD_TIPO_INVERSION,ID_PROYECTO,DIFF_MONTO,DIFF_PORC_MONTO,NOW,DIFF_FECHA_INI_EJE_ET,DIFF_FECHA_NOW_ET,DIFF_FECHA_NOW_INI_EJE,NCOR_LONGIT,NCOR_LATITU,NCOR_LONGI2,NCOR_LATIT2,CDPT_CD_DPTO,CDPT_DPTO,CDPT_DPTO.1,CPRV_PROV,CDST_CODIGO,CDST_DISTRIT,DOBA_FCHREP_count,DOBA_FCHREP_min,DOBA_FCHREP_max,NOBA_FISICO_PRO_min,NOBA_FISICO_PRO_max,NOBA_FISICO_REA_min,NOBA_FISICO_REA_max,NOBA_VALZDO_PRO_min,NOBA_VALZDO_PRO_max,NOBA_VALZDO_REA_min,NOBA_VALZDO_REA_max,Monto actualizado S/_count,Monto actualizado S/_min,Monto actualizado S/_max
0,47664,CONTRATACIÓN DE LA FIRMA CONTRATISTA QUE EJECU...,661,El Tramo I del proyecto se inicia en Bellavist...,202,Por Contrata,TRANSPORTES Y COMUNICACIONES,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,AUGUSTO ALEJANDRO. LLUEN FLORES,D.N.I: 06261463,/Date(1511499600000)/,396,/Date(1430974800000)/,2.142165e+09,/Date(1403067600000)/,1.128900e+09,2192666,CONSTRUCCION DE LA CARRETERA BELLAVISTA - MAZA...,2192666,2022-04-24,2017-11-24 05:00:00.000000,0.0,AGRICULTURA Y RIEGO,MINISTERIO DE AGRICULTURA Y RIEGO - MINAGRI,TRANSPORTE,11114703.0,9563.78,6.716139e+08,GN,39183.0,dirección de puentes de pvn,PROYECTO ESPECIAL BINACIONAL DESARROLLO INTEGR...,1.542915e+09,1.128900e+09,CONSTRUCCION DE LA CARRETERA BELLAVISTA - MAZA...,2001-07-26 09:57:33.000000,2014-06-18 05:00:00.000000,PROYECTO DE INVERSION,2014-05,2022-03,6627862.76,9563.78,12923.78,2988777.0,42023766.0,0.0,0.0,0.0,OPMI DEL MINISTERIO DE AGRICULTURA - MINAG,ACTIVO,2015-04-30 05:00:00.000000,NO,2017-07-18 00:00:00.000000,2024-03-27 00:00:00.000000,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,SNIP,396.0,SI,TRANSPORTE TERRESTRE,CONSTRUCCION Y MEJORAMIENTO DE CARRETERAS,VIABLE,121641.0,0.0,0.0,4.140149e+08,0.366742,2022-05-06 05:35:36.049004,2.700000e+01,85.0,58.0,-72.952480,-3.450366,-73.067665,-3.549744,16.0,LORETO,LORETO,MAYNAS,6.0,MAZAN,48.0,2017-12-20 16:28:55.000000,2021-12-17 19:22:45.000000,0.01,100.00,0.02,96.19,38544.81,5.097766e+08,107528.15,4.903483e+08,4.0,6.449693e+08,1.542915e+09
1,159977,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,4382,MARGEN IZQUIERDO DEL RIO CORRIENTES,207,Nucleos Ejecutores,TRANSPORTES Y COMUNICACIONES,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL,NaN,NaN,/Date(1636952400000)/,2506945,/Date(-62135578800000)/,2.008550e+05,/Date(1608094800000)/,2.033030e+05,2506945,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,2506945,2022-04-24,2021-11-15 05:00:00.000000,22403.0,DESARROLLO E INCLUSION SOCIAL,MINISTERIO DE DESARROLLO E INCLUSION SOCIAL - ...,TRANSPORTE,0.0,0.00,0.000000e+00,GN,27634.0,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,2.008549e+05,2.033026e+05,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,2020-12-15 03:04:42.000000,2020-12-16 05:10:55.000000,IOARR,NaN,NaN,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,OPMI DEL MINISTERIO DE DESARROLLO E INCLUSION ...,ACTIVO,2021-07-21 05:00:00.000000,NO,2021-01-25 00:00:00.000000,2021-11-07 00:00:00.000000,ADMINISTRACIÓN INDIRECTA - NÚCLEO EJECUTOR,INVIERTE,2506945.0,NO,TRANSPORTE TERRESTRE,CAMINOS DE HERRADURA,APROBADO,0.0,2.0,117416.0,-2.447720e+03,-0.012040,2022-05-06 05:35:36.049004,-6.000000e+00,10.0,16.0,-75.144327,-3.

In [25]:
datos['RN'] = datos.sort_values(['COSTO_ACTUALIZADO', 'MTO_VIABLE'], ascending=[False, False]) \
             .groupby(['NOBR_ID']) \
             .cumcount() + 1

In [26]:
datos = datos[datos['RN']==1].drop('RN', axis=1).copy()

In [27]:
datos['RN'] = datos.sort_values(['COSTO_ACTUALIZADO', 'MTO_VIABLE'], ascending=[False, False]) \
             .groupby(['CODIGO_UNICO']) \
             .cumcount() + 1

In [28]:
datos = datos[datos['RN']==1].drop('RN', axis=1).copy()

In [29]:
datos['NOBR_ID'].shape, datos['NOBR_ID'].nunique(), datos['CODIGO_UNICO'].nunique()

((15092,), 15092, 15092)

In [30]:
datos.to_csv('../Datos/Matrices/Matriz_infobra_unicos.txt', sep='|', index=False)

In [31]:
datos = pd.read_csv('../Datos/Matrices/Matriz_infobra_unicos.txt', sep='|')

In [32]:
datos['NOBR_ID'].shape, datos['NOBR_ID'].nunique()

((15092,), 15092)

In [33]:
datos['NOBR_ID'].shape, datos['NOBR_ID'].nunique(), datos['CODIGO_UNICO'].nunique()

((15092,), 15092, 15092)

In [37]:
datos.head(2)

,NOBR_ID,COBR_DESCRI,CODCONSUCODE,COBR_DIRECCION,NOBR_TIPEJE,COPA_DESCRI,RUBRO,CODCONSUCODE_DESCRI,COBR_NOMBRERESIDENTE,COBR_DNIRESIDENTE,DOBR_FEC_INI_OBRA,NOBR_CODSNIP,DSNP_FECVERI,NSNP_MONTOVERI,DSNP_FECAPR,NSNP_MONTOAPR,CSNP_CODSIAF,CSNP_DESCRI,CODIGO_UNICO,FECHA_ACTUALIZACION,FECHA_INICIO,UNIDAD_OPMI,SECTOR,ENTIDAD,FUNCION,PIM_ANO_VIGENTE,DEV_ANO_VIGENTE,DEV_ACUMULADO,NIVEL,COD_UNIDAD_UEI,DES_UNIDAD_UEI,DES_UNIDAD_UF,COSTO_ACTUALIZADO,MTO_VIABLE,NOMBRE_INVERSION,FEC_REGISTRO,FEC_VIABLE,TIPO_FORMATO,MES_ANO_PRI_DEV,MES_ANO_ULT_DEV,CERTIFICADO_ANO_ACTUAL,GIRADO_ANO_ACTUAL,COMPROMETIDO_ANO_ACTUAL,MTO_PMI_1,MTO_PMI_2,MTO_PMI_3,MTO_PMI_4,MTO_CartFza,NOMBRE_OPMI,ESTADO,FECHA_ET,CIERRE_REGISTRADO,FEC_INI_EJ,FEC_FIN_EJ,MODAL_EJEC,MARCO,COD_SNIP,IND_REG_PMI,DES_PROGRAMA,DES_SUB_PROGRAMA,SITUACION,BENEFICIARIO,COD_TIPO_INVERSION,ID_PROYECTO,DIFF_MONTO,DIFF_PORC_MONTO,NOW,DIFF_FECHA_INI_EJE_ET,DIFF_FECHA_NOW_ET,DIFF_FECHA_NOW_INI_EJE,NCOR_LONGIT,NCOR_LATITU,NCOR_LONGI2,NCOR_LATIT2,CDPT_CD_DPTO,CDPT_DPTO,CDPT_DPTO.1,CPRV_PROV,CDST_CODIGO,CDST_DISTRIT,DOBA_FCHREP_count,DOBA_FCHREP_min,DOBA_FCHREP_max,NOBA_FISICO_PRO_min,NOBA_FISICO_PRO_max,NOBA_FISICO_REA_min,NOBA_FISICO_REA_max,NOBA_VALZDO_PRO_min,NOBA_VALZDO_PRO_max,NOBA_VALZDO_REA_min,NOBA_VALZDO_REA_max,Monto actualizado S/_count,Monto actualizado S/_min,Monto actualizado S/_max
0,47664,CONTRATACIÓN DE LA FIRMA CONTRATISTA QUE EJECU...,661,El Tramo I del proyecto se inicia en Bellavist...,202,Por Contrata,TRANSPORTES Y COMUNICACIONES,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,AUGUSTO ALEJANDRO. LLUEN FLORES,D.N.I: 06261463,/Date(1511499600000)/,396,/Date(1430974800000)/,2.142165e+09,/Date(1403067600000)/,1.128900e+09,2192666,CONSTRUCCION DE LA CARRETERA BELLAVISTA - MAZA...,2192666,2022-04-24,2017-11-24 05:00:00.000000,0.0,AGRICULTURA Y RIEGO,MINISTERIO DE AGRICULTURA Y RIEGO - MINAGRI,TRANSPORTE,11114703.0,9563.78,6.716139e+08,GN,39183.0,dirección de puentes de pvn,PROYECTO ESPECIAL BINACIONAL DESARROLLO INTEGR...,1.542915e+09,1.128900e+09,CONSTRUCCION DE LA CARRETERA BELLAVISTA - MAZA...,2001-07-26 09:57:33.000000,2014-06-18 05:00:00.000000,PROYECTO DE INVERSION,2014-05,2022-03,6627862.76,9563.78,12923.78,2988777.0,42023766.0,0.0,0.0,0.0,OPMI DEL MINISTERIO DE AGRICULTURA - MINAG,ACTIVO,2015-04-30 05:00:00.000000,NO,2017-07-18 00:00:00.000000,2024-03-27 00:00:00.000000,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,SNIP,396.0,SI,TRANSPORTE TERRESTRE,CONSTRUCCION Y MEJORAMIENTO DE CARRETERAS,VIABLE,121641.0,0.0,0.0,4.140149e+08,0.366742,2022-05-06 05:35:36.049004,27.0,85.0,58.0,-72.952480,-3.450366,-73.067665,-3.549744,16.0,LORETO,LORETO,MAYNAS,6.0,MAZAN,48.0,2017-12-20 16:28:55.000000,2021-12-17 19:22:45.000000,0.01,100.00,0.02,96.19,38544.81,5.097766e+08,107528.15,4.903483e+08,4.0,6.449693e+08,1.542915e+09
1,159977,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,4382,MARGEN IZQUIERDO DEL RIO CORRIENTES,207,Nucleos Ejecutores,TRANSPORTES Y COMUNICACIONES,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL,NaN,NaN,/Date(1636952400000)/,2506945,/Date(-62135578800000)/,2.008550e+05,/Date(1608094800000)/,2.033030e+05,2506945,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,2506945,2022-04-24,2021-11-15 05:00:00.000000,22403.0,DESARROLLO E INCLUSION SOCIAL,MINISTERIO DE DESARROLLO E INCLUSION SOCIAL - ...,TRANSPORTE,0.0,0.00,0.000000e+00,GN,27634.0,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,FONDO DE COOPERACION PARA EL DESARROLLO SOCIAL...,2.008549e+05,2.033026e+05,CONSTRUCCION DE DISPOSITIVO DE CONTROL DE TRAN...,2020-12-15 03:04:42.000000,2020-12-16 05:10:55.000000,IOARR,NaN,NaN,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,OPMI DEL MINISTERIO DE DESARROLLO E INCLUSION ...,ACTIVO,2021-07-21 05:00:00.000000,NO,2021-01-25 00:00:00.000000,2021-11-07 00:00:00.000000,ADMINISTRACIÓN INDIRECTA - NÚCLEO EJECUTOR,INVIERTE,2506945.0,NO,TRANSPORTE TERRESTRE,CAMINOS DE HERRADURA,APROBADO,0.0,2.0,117416.0,-2.447720e+03,-0.012040,2022-05-06 05:35:36.049004,-6.0,10.0,16.0,-75.144327,-3.811037,0.000000,0